In [6]:
!pip install transformers


In [7]:
!pip install -q tf-models-official==2.3.0
import json

In [8]:
with open("training.en-en.data", "r",encoding='UTF-8') as read_file:
    data = json.load(read_file)
with open("training.en-en.gold", "r",encoding='UTF-8') as read_file:
    gold = json.load(read_file)
with open("dev.en-en.data", "r",encoding='UTF-8') as read_file:
    dev_data = json.load(read_file)
with open("dev.en-en.gold", "r",encoding='UTF-8') as read_file:
    dev_gold = json.load(read_file)
with open("test.en-en.data", "r",encoding='UTF-8') as read_file:
    test_data = json.load(read_file)
with open("test.en-en.gold", "r",encoding='UTF-8') as read_file:
    test_gold = json.load(read_file)

In [9]:
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize   
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))   

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def map_pos(wordnet,pos):
  if pos=='NOUN':
    return wordnet.NOUN
  elif pos=='VERB':
    return wordnet.VERB
  elif pos=='ADJ':
    return wordnet.ADJ
  elif pos=='ADV':
    return wordnet.ADV
  else:
    return pos

In [12]:
def sort ( dict1):
  sorted_dict = {}
  sorted_values = sorted(dict1.values()) 
  for i in sorted_values:
      for k in dict1.keys():
          if dict1[k] == i:
              sorted_dict[k] = dict1[k]
              break
  return sorted_dict

In [13]:
def get_wordnet_results(data):
  sents1_path=[]
  sents2_path=[]
  punctuations = '''!()-—[]{};:'”"\,<>./?@#$%^&*_~'''
  count=0
  for dat in data:
    sent1=dat['sentence1']
    sent2=dat['sentence2']
    wordInFirst=sent1[int(dat['start1']):int(dat['end1'])].lower()
    wordInSecond=sent2[int(dat['start2']):int(dat['end2'])].lower()
    pos=dat['pos']
    lemma=dat['lemma']
    for ch in punctuations:
      sent1= sent1.lower().replace(ch,' ')
    for ch in punctuations:
      sent2= sent2.lower().replace(ch,' ')
    # print(sent1)
    word_tokens1 = word_tokenize(sent1.lower())  
    word_tokens2 = word_tokenize(sent2.lower())  
    sent1 = [w for w in word_tokens1 if w in wordInFirst or (not w in stop_words )] 
    sent2 = [w for w in word_tokens2 if w in wordInSecond or ( not w in stop_words)]
    pos= pos.split("\t")[0]
    # print(pos)
    pos_wn= map_pos(wordnet,pos)
    # print(sent1)
    # print(sent2)
    # print(wordInSecond)
    if count%1000==0:
      print(count)
    count+=1
    result1=find_paths(sent1,lemma,wordInFirst,pos_wn)
    result2=find_paths(sent2,lemma,wordInSecond,pos_wn)
    result1= sort(result1)
    result2= sort(result2)
    sents1_path.append(result1)
    sents2_path.append(result2)
  return sents1_path,sents2_path

In [14]:
import numpy as np
def find_paths(words, lemma , w, pos):
  ind =words.index(w)
  syns = wordnet.synsets(lemma,pos)
  result={}
  for syn in syns:
    # print(syn.lexname())
    sum=0
    num=0
    for i in range(len(words)):
      word=words[i]
      if i!=ind:
        syns2= wordnet.synsets(word,pos)
        # print(syns2)
        min=10000
        for syn2 in syns2:
          path= syn.shortest_path_distance(syn2)
          if path!=None:
            if path<min:
              min=path
        if min<10000:
          num=(np.abs(i-ind))
          sum+=min*(num)
    if num!=0:
      result[syn.lexname()]=sum
  return result


In [15]:
labels_tmp= np.array([dat['tag'] for dat in gold])
labels= [0 if x=='F' else 1 for x in labels_tmp]
labels=np.asarray(labels)

In [16]:
labels_tmp_val= np.array([dat['tag'] for dat in dev_gold])
labels_val= [0 if x=='F' else 1 for x in labels_tmp_val]
labels_val=np.asarray(labels_val)

In [17]:
labels_tmp_test= np.array([dat['tag'] for dat in test_gold])
labels_test= [0 if x=='F' else 1 for x in labels_tmp_test]
labels_test=np.asarray(labels_test)

In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
from sklearn.model_selection import train_test_split
import json
# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks
import pandas as pd
import transformers as ppb
from nltk.corpus import wordnet

hub_url_bert = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# tokenizer =tokenizer_class.from_pretrained(pretrained_weights)
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

print("Vocab size:", len(tokenizer.vocab))
def merge(lst1, lst2):
    return [a + b[1:] for (a, b) in zip(lst1, lst2)]

def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)
# df = pd.DataFrame(columns = 'label',data=[dat['tag'] for dat in gold])



Vocab size: 30522


In [19]:
def prepareData(data):
    sentence1=[]
    sentence2=[]
    paraphrased1_encode=[]
    paraphrased2_encode=[]
    # lemmas1=[]
    # lemmas2=[]
    types_s1=[]
    types_s2=[]
    result_wordnet1,result_wordnet2=get_wordnet_results(data)
    count=0
    for dat in data:
        sent1=dat['sentence1']
        sent2=dat['sentence2']
        lemma=dat['lemma']
        wordInFirst=sent1[int(dat['start1']):int(dat['end1'])]
        wordInSecond=sent2[int(dat['start2']):int(dat['end2'])]
        newSent1=sent1[0:int(dat['start1'])]+' [TGT] '+wordInFirst+' [TGT] '+sent1[int(dat['end1']):len(sent1)]
        newSent2=sent2[0:int(dat['start2'])]+' [TGT] '+ wordInSecond+' [TGT] '+sent2[int(dat['end2']):len(sent2)]
        sent1_wordnet=''
        sent2_wordnet=''
        for lexname in result_wordnet1[count]:
          sent1_wordnet+=' [LX] '+lexname.split('.')[1]
        for lexname in result_wordnet2[count]:
          sent2_wordnet+=' [LX] '+lexname.split('.')[1]
        newSent1=newSent1+sent1_wordnet
        newSent2=newSent2+sent2_wordnet

        # print(newSent1)
        # print(newSent2)

        encoded_sent1=encode_sentence(newSent1)
        encoded_sent2=encode_sentence(newSent2)

        # print(encoded_sent1)
        # print(encoded_sent2)
        sentence1.append(encoded_sent1)
        sentence2.append(encoded_sent2)
        encode_w1= encode_sentence(wordInFirst)
        encode_w2= encode_sentence(wordInSecond)
        count+=1

    sentence1=tf.ragged.constant(sentence1)
    sentence2=tf.ragged.constant(sentence2)

    # paraphrased1_encode=tf.ragged.constant(paraphrased1_encode)
    # paraphrased2_encode=tf.ragged.constant(paraphrased2_encode)

    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])] * sentence1.shape[0]
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)
    # input_para_ids = tf.concat([cls, paraphrased1_encode, paraphrased2_encode], axis=-1)

    input_mask = tf.ones_like(input_word_ids).to_tensor()
    type_cls = tf.zeros_like(cls)
    types_s1 = tf.ones_like(sentence1)
    types_s2 = tf.ones_like(sentence2)
    types_s2 = types_s2+1;
    input_type_ids = tf.concat([type_cls, types_s1, types_s2], axis=-1).to_tensor()
    inputs = {
        'input_word_ids': input_word_ids.to_tensor(),
        # 'input_para_ids': input_para_ids.to_tensor(),
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs

In [20]:
inputsTrain=prepareData(data)

0
1000
2000
3000
4000
5000
6000
7000


In [21]:
inputsVal=prepareData(dev_data)

0


In [22]:
inputsTest=prepareData(test_data)

0


In [23]:
lemmas1=[]
lemmas2=[]
for dat in data:
  sent1=dat['sentence1']
  sent2=dat['sentence2']
  encoded_sent1=encode_sentence(sent1)
  encoded_sent2=encode_sentence(sent2)
  print(sent1)
  print(encoded_sent1)
  print(sent2)
  print(encoded_sent2)
  encode_lemma= encode_sentence('play')
  lemmas1.append(encoded_sent1.index(encode_lemma[0]))
  lemmas2.append(encoded_sent2.index(encode_lemma[0]))
  break
print(lemmas1)
print(lemmas2)

In that context of coordination and integration, Bolivia holds a key play in any process of infrastructure development.
[1999, 2008, 6123, 1997, 12016, 1998, 8346, 1010, 11645, 4324, 1037, 3145, 2377, 1999, 2151, 2832, 1997, 6502, 2458, 1012, 102]
A musical play on the same subject was also staged in Kathmandu for three days.
[1037, 3315, 2377, 2006, 1996, 2168, 3395, 2001, 2036, 9813, 1999, 28045, 2005, 2093, 2420, 1012, 102]
[12]
[2]


In [ ]:

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

print(config_dict)
# config_dict={'attention_probs_dropout_prob': 0.1,
#  'hidden_act': 'relu',
#  'hidden_dropout_prob': 0.1,
#  'hidden_size': 768,
#  'initializer_range': 0.02,
#  'intermediate_size': 3072,
#  'max_position_embeddings': 512,
#  'num_attention_heads': 12,
#  'num_hidden_layers': 12,
#  'type_vocab_size': 2,
#  'vocab_size': 30522}

# bert_config = bert.configs.BertConfig.from_dict(config_dict)

# print('test')
# print(bert_config['initializer_range'])
# print(bert.bert_models.get_transformer_encoder( bert_config ,sequence_length=100))
bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=2)
tf.keras.utils.plot_model(bert_classifier, show_shapes=True, dpi=48)
checkpoint = tf.train.Checkpoint(model=bert_encoder)
checkpoint.restore(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()
# print(bert_encoder.get_embedding_table())
epochs = 10
batch_size = 16
eval_batch_size = 16

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=2,
                                                    mode='min')
train_data_size = len(labels)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bert_classifier.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

bert_classifier.fit(
      inputsTrain, labels,
      validation_data=(inputsVal,labels_val),
      callbacks=[early_stopping],
      batch_size=16,
      epochs=epochs)
export_dir='./saved_model'
tf.saved_model.save(bert_classifier, export_dir=export_dir)

{'attention_probs_dropout_prob': 0.1, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 30522}
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


INFO:absl:using Adamw optimizer


Epoch 1/10
370/500 [=====================>........] - ETA: 3:13 - loss: 0.7158 - accuracy: 0.5338

In [ ]:
bert_classifier_model = tf.keras.models.load_model('drive/MyDrive/saved_model/')


In [ ]:
result= bert_classifier.predict(inputsTest)
result = tf.argmax(result,axis=1).numpy()
test_result = []
for i in range(len(result)):
  id="test.en-en.{}".format(i)
  test_result.append({
      'id':id,
      'tag':'T' if result[i]==1 else 'F'
  })
json_object = json.dumps(test_result, indent = 4) 
with open('test.en-en', 'w') as outfile:
    outfile.write(json_object) 

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
inputs_test=prepareData(test_data)
result= bert_classifier.predict(inputs_test)
result = tf.argmax(result,axis=1).numpy()
# print(test_labels.shape)
print(accuracy_score(labels_test,result))

0
0.855


In [ ]:
result= bert_classifier(inputsTest,training=False)
result = tf.argmax(result,axis=1).numpy()
# print(test_labels.shape)
print(classification_report(test_labels,result))
